In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False)
model.eval()

In [21]:
def remove_suffix(input_string, suffix):
    if suffix and input_string.endswith(suffix):
        return input_string[:-len(suffix)]
    return input_string

In [23]:
from collections import defaultdict
import csv
import glob
import os 

# os.chdir(r"Plant_Phenotyping_Datasets/Plant/Ara2013-RPi") # directory in question

# construct dictionary of labels
# Metadata.csv contains labels with keys ara2013_plant000 to ara2013_plant165
label = defaultdict(lambda: "unknown")
with open("Metadata.csv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    for i, line in enumerate(reader):
        line = line[0].split(',') # split string into categories
        line[1] = line[1].lstrip() # clear whitespace
        label[line[0]] = line[1]

# image names ara2013_plant000_rgb.png to ara2013_plant165_rgb.png
for img_filename in glob.glob("*.png"):
    name = remove_suffix(img_filename, "_rgb.png")
    # to access the genotype, use label[name]
    


defaultdict(<function <lambda> at 0x7f0a10738d40>, {'ara2013_plant001': 'ein2', 'ara2013_plant002': 'ein2', 'ara2013_plant003': 'ctr', 'ara2013_plant004': 'Col-0', 'ara2013_plant005': 'pgm', 'ara2013_plant006': 'ein2', 'ara2013_plant007': 'ctr', 'ara2013_plant008': 'ein2', 'ara2013_plant009': 'ctr', 'ara2013_plant010': 'ctr', 'ara2013_plant011': 'adh1', 'ara2013_plant012': 'ein2', 'ara2013_plant013': 'pgm', 'ara2013_plant014': 'ein2', 'ara2013_plant015': 'ctr', 'ara2013_plant016': 'Col-0', 'ara2013_plant017': 'ctr', 'ara2013_plant018': 'adh1', 'ara2013_plant019': 'ein2', 'ara2013_plant020': 'ctr', 'ara2013_plant021': 'Col-0', 'ara2013_plant022': 'pgm', 'ara2013_plant023': 'Col-0', 'ara2013_plant024': 'ein2', 'ara2013_plant025': 'ctr', 'ara2013_plant026': 'adh1', 'ara2013_plant027': 'pgm', 'ara2013_plant028': 'ein2', 'ara2013_plant029': 'ctr', 'ara2013_plant030': 'pgm', 'ara2013_plant031': 'Col-0', 'ara2013_plant032': 'ein2', 'ara2013_plant033': 'ctr', 'ara2013_plant034': 'adh1', 'ara20